In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# KFold (교차 검증을 사용하기 위해)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가 함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRFRegressor

# 저장
import pickle

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우용
# plt.rcParams['font.family'] = 'AppleGothic'   # 맥용
plt.rcParams['font.size'] = 16                  # 폰트 크기
plt.rcParams['figure.figsize'] = 10,8          # 그래프 크기
plt.rcParams['axes.unicode_minus'] = False     # - 기호 깨짐 방지


# 경고 메시지가 안나오게 하기
import warnings
warnings.filterwarnings('ignore')

## 데이터 불러오기

In [3]:
cctv = pd.read_csv('../../원본데이터/CCTV 현황/센터보유 ITS CCTV 조회.csv',encoding='cp949')
cctv.head()

,NAME,CCTV_X,CCTV_Y,ID,URL
0,센텀중,129.123994,35.173625,79,https://its.busan.go.kr/traffic/exclude/cctvPo...
1,수영1호교사거리,129.125610,35.165390,53,https://its.busan.go.kr/traffic/exclude/cctvPo...
2,수영2호교입구(민락),129.128130,35.159910,54,https://its.busan.go.kr/traffic/exclude/cctvPo...
3,하마정교차로,129.067785,35.174770,47,https://its.busan.go.kr/traffic/exclude/cctvPo...
4,송공(광장),129.068768,35.169229,14,https://its.busan.go.kr/traffic/exclude/cctvPo...


### 결측치 확인

In [5]:
cctv.isna().sum()

NAME      0
CCTV_X    0
CCTV_Y    0
ID        0
URL       0
dtype: int64

### 중복값제거

---

## 교회

In [6]:
church = pd.read_csv('../../원본데이터/부산시 교회/교회_위경도.csv',encoding='cp949')
church.head()

,Column1,church_name,church_lot,Latitude,Longitude
0,115,가야제일교회,가야동 13-67,35.155289,129.044055
1,60,영안교회 가야예배당,가야동 150-9,35.151764,129.029545
2,186,가야교회 가야성전,가야동 345-2,35.152427,129.036348
3,204,감만교회,감만동 130-1,35.115846,129.079964
4,357,동항교회,감만동 90-2,35.117243,129.080253


### 결측치 확인

In [7]:
church.isna().sum()

Column1        0
church_name    0
church_lot     0
Latitude       2
Longitude      2
dtype: int64

2개의 컬럼의 위도,경도 결측값

### 중복값제거

---

## 학원

In [10]:
acad = pd.read_csv('../../원본데이터/학원, 학교, 유치원 위치 현황/부산 학원, 교습소 현황 - 학원명단 _ 위경도.csv',encoding='cp949')
acad.head()

,연번,관할교육청명,구분,학원명,계열,교습과정,학원주소,Latitude,Longitude,우편번호,전화번호,Unnamed: 11,Unnamed: 12
0,1,서부,학교교과교습학원,원탑영수학원,보통교과,보습,"부산광역시 영도구 와치로 252 , 3층 일부 (동삼동, 선프라자)",35.075353,129.069017,49115.0,NaN,NaN,NaN
1,2,서부,학교교과교습학원,연피아노학원,예능,음악,"부산광역시 서구 대영로38번길 11 , 109동 201호 전부 (서대신동1가, 대신...",35.108263,129.015886,49232.0,NaN,NaN,NaN
2,3,서부,학교교과교습학원,플랜에이영어학원,보통교과,보습,"부산광역시 사하구 낙동대로 527 , 3층 전부 (하단동)",35.111069,128.964821,49308.0,051-710-3111,NaN,NaN
3,5,서부,학교교과교습학원,해강음악학원,예능,음악,"부산광역시 사하구 제석로 49 , 2층전부 (당리동)",35.105355,128.974071,49326.0,051-292-4336,NaN,NaN
4,6,서부,학교교과교습학원,아름음악학원,예능,음악,"부산광역시 사하구 승학로 128 (당리동, 무궁화빌라)",35.103534,128.981458,49331.0,051-203-3513,NaN,NaN


### 결측치 확인

In [11]:
acad.isna().sum()

연번                0
관할교육청명            0
구분                0
학원명               0
계열              136
교습과정            135
학원주소              0
Latitude          2
Longitude         2
우편번호              3
전화번호           1136
Unnamed: 11    4626
Unnamed: 12    4625
dtype: int64

2개의 컬럼의 위도,경도 결측값 제거 <br>
Unnamed: 11,Unnamed: 12  컬럼 제거

### 중복값제거